# Adv Games Regression
<a id="top"></a>

Treating games as the sum of each team's players stats.
0. [Helper Methods](#helpers)
1. [Score Differential Regressor](#score-diff)
    * [SVD regression](#score-diff-lr), [RFT regression](#score-diff-logr)
    * Only take raw game scores and betting lines
2. [Past Win% Differential Regressor](#winper)
    * [SVD regression](#winper-lr), [RFT regression](#winper-logr)
    * Take past win% over 10, 25, 50, 75, 100 games for each team
3. [Box Score Regressor](#boxscore)
    * Takes in overall box score for either team 

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.decomposition import PCA

In [5]:
pd.set_option('display.max_columns', 250)
pd.options.display.float_format = '{:5}'.format

## Helper Methods
[to top](#top)
<a id="helpers">

In [18]:
def SVGRegression(X_input, y_input):
    n_folds = 5
    kf = KFold(n_folds, shuffle=True)

    # Create a pipeline that first standardizes the data, then applies PCA, and finally trains the model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components="mle")),  # k is the number of principal components
        ('regressor', SVR())
    ])

    # Perform cross-validation
    cv_scores = cross_val_score(pipeline, X_input, np.ravel(y_input), cv=kf, scoring='neg_mean_squared_error')

    # Calculate average MSE
    average_mse = -np.mean(cv_scores)
    print("cv:", cv_scores)
    print("Average MSE:", average_mse)
    print("Average RMSE:", np.sqrt(average_mse))

    return pipeline

def SVGRegressionFit(X_input, y_input):

    X_train, X_test, y_train, y_test = train_test_split(X_input, y_input, test_size=.2)

    # Create a pipeline that first standardizes the data, then applies PCA, and finally trains the model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components="mle")),  # k is the number of principal components
        ('regressor', SVR())
    ])

    pipeline.fit(X_train, np.ravel(y_train))
    # y_pred = pipeline.predict(X_test)

    # # The coefficients
    # # The mean squared error
    # print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
    # # The coefficient of determination: 1 is perfect prediction
    # print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

    return pipeline

## Rolling Regressors
[to top](#top)

-  [linear regression](#winper-lr)
-  [logistic regression](#winper-logr)
<a id="winper">

In [16]:
df = pd.read_csv("data_proc/games_rolling_master.csv")
df.dropna(inplace=True)
# display(df)
game_info = ['TEAM_ID_T1', 'TEAM_ID_T2', 'DATE', 'away_prob', 'away_spread', 'away_total'] 
intervals = ['10g_', '25g_', '50g_', '75g_', '100g']
columns = game_info + [c for c in df.columns.to_list() if c not in game_info and c[:4] in intervals]
print(columns)

['TEAM_ID_T1', 'TEAM_ID_T2', 'DATE', 'away_prob', 'away_spread', 'away_total', '10g_FGM_T1', '10g_FGM_T2', '10g_FGA_T1', '10g_FGA_T2', '10g_FG3M_T1', '10g_FG3M_T2', '10g_FG3A_T1', '10g_FG3A_T2', '10g_FTM_T1', '10g_FTM_T2', '10g_FTA_T1', '10g_FTA_T2', '10g_OREB_T1', '10g_OREB_T2', '10g_DREB_T1', '10g_DREB_T2', '10g_REB_T1', '10g_REB_T2', '10g_AST_T1', '10g_AST_T2', '10g_STL_T1', '10g_STL_T2', '10g_BLK_T1', '10g_BLK_T2', '10g_TO_T1', '10g_TO_T2', '10g_PF_T1', '10g_PF_T2', '10g_PTS_T1', '10g_PTS_T2', '10g_PLUS_MINUS_T1', '10g_PLUS_MINUS_T2', '10g_FG_PCT_T1', '10g_FG_PCT_T2', '10g_FG3_PCT_T1', '10g_FG3_PCT_T2', '10g_FT_PCT_T1', '10g_FT_PCT_T2', '10g_SCORE_DIFF_T1', '10g_SCORE_DIFF_T2', '10g_WON_T1', '10g_WON_T2', '25g_FGM_T1', '25g_FGM_T2', '25g_FGA_T1', '25g_FGA_T2', '25g_FG3M_T1', '25g_FG3M_T2', '25g_FG3A_T1', '25g_FG3A_T2', '25g_FTM_T1', '25g_FTM_T2', '25g_FTA_T1', '25g_FTA_T2', '25g_OREB_T1', '25g_OREB_T2', '25g_DREB_T1', '25g_DREB_T2', '25g_REB_T1', '25g_REB_T2', '25g_AST_T1', '25g_AS

In [31]:
reg = SVGRegressionFit(df.iloc[:-1000][columns].values, df[:-1000]["SCORE_DIFF_T1"].values)
                
# SVGRegression(df[['TEAM1_ID', 'TEAM2_ID', 'DATE', 'TEAM1_10wp', 'TEAM1_25wp', 'TEAM1_50wp', 'TEAM1_75wp', 'TEAM1_100wp', 'TEAM2_10wp', 'TEAM2_25wp', 'TEAM2_50wp', 'TEAM2_75wp', 'TEAM2_100wp']],df[["SCORE_DIFF_T1"]])

# SVGRegression(df.drop(columns=['SCORE_DIFF_T1', 'SCORE_DIFF_T2', '']), df[["SCORE_DIFF_T1"]])
# fig = px.scatter(df, x="away_spread", y="SCORE_DIFF", hover_data=["SBR_GAME_ID"], trendline="ols")
# fig.show()


In [32]:
def testIndex(i):
    print(f"id : {df.iloc[-i]['SBR_GAME_ID']}")
    print(f"spread : {df.iloc[-i]['away_spread']}")
    print(f"predict: {reg.predict([df.iloc[-i][columns].values])}")
    print(f"actual : {df.iloc[-i]['SCORE_DIFF_T1']}")
    print()

for i in range(100):
    testIndex(i)

id : 20160301_nba_Atlanta_Golden State
spread : 5.5
predict: [-5.5172657]
actual : -4.0

id : 20231110_nba_New Orleans_Houston
spread : 3.0
predict: [-0.10133272]
actual : -3.0

id : 20231110_nba_Oklahoma City_Sacramento
spread : -1.75
predict: [0.38798438]
actual : -7.0

id : 20231110_nba_Utah_Memphis
spread : 5.0
predict: [-3.83296266]
actual : 6.0

id : 20231110_nba_L.A. Clippers_Dallas
spread : 1.0
predict: [-1.6988918]
actual : -18.0

id : 20231110_nba_Philadelphia_Detroit
spread : -8.5
predict: [6.32337352]
actual : 8.0

id : 20231110_nba_Minnesota_San Antonio
spread : -7.25
predict: [5.5913872]
actual : 7.0

id : 20231110_nba_Brooklyn_Boston
spread : 12.0
predict: [-6.78187535]
actual : -14.0

id : 20231110_nba_L.A. Lakers_Phoenix
spread : 2.0
predict: [-2.43553993]
actual : 3.0

id : 20231110_nba_Charlotte_Washington
spread : 3.0
predict: [-2.95873445]
actual : 7.0

id : 20231109_nba_Atlanta_Orlando
spread : -3.0
predict: [1.16919296]
actual : 1.0

id : 20231109_nba_Milwaukee_I

In [34]:
df = pd.read_csv("data_proc/games_rolling_master.csv")
df

,GAME_ID,SBR_GAME_ID,DATE,TEAM_ID_T1,FGM_T1,FGA_T1,FG3M_T1,FG3A_T1,FTM_T1,FTA_T1,OREB_T1,DREB_T1,REB_T1,AST_T1,STL_T1,BLK_T1,TO_T1,PF_T1,PTS_T1,PLUS_MINUS_T1,FG_PCT_T1,FG3_PCT_T1,FT_PCT_T1,SCORE_DIFF_T1,WON_T1,TEAM_ID_T2,FGM_T2,FGA_T2,FG3M_T2,FG3A_T2,FTM_T2,FTA_T2,OREB_T2,DREB_T2,REB_T2,AST_T2,STL_T2,BLK_T2,TO_T2,PF_T2,PTS_T2,PLUS_MINUS_T2,FG_PCT_T2,FG3_PCT_T2,FT_PCT_T2,SCORE_DIFF_T2,WON_T2,away_prob,away_spread,away_total,10g_FGM_T1,10g_FGM_T2,10g_FGA_T1,10g_FGA_T2,10g_FG3M_T1,10g_FG3M_T2,10g_FG3A_T1,10g_FG3A_T2,10g_FTM_T1,10g_FTM_T2,10g_FTA_T1,10g_FTA_T2,10g_OREB_T1,10g_OREB_T2,10g_DREB_T1,10g_DREB_T2,10g_REB_T1,10g_REB_T2,10g_AST_T1,10g_AST_T2,10g_STL_T1,10g_STL_T2,10g_BLK_T1,10g_BLK_T2,10g_TO_T1,10g_TO_T2,10g_PF_T1,10g_PF_T2,10g_PTS_T1,10g_PTS_T2,10g_PLUS_MINUS_T1,10g_PLUS_MINUS_T2,10g_FG_PCT_T1,10g_FG_PCT_T2,10g_FG3_PCT_T1,10g_FG3_PCT_T2,10g_FT_PCT_T1,10g_FT_PCT_T2,10g_SCORE_DIFF_T1,10g_SCORE_DIFF_T2,10g_WON_T1,10g_WON_T2,25g_FGM_T1,25g_FGM_T2,25g_FGA_T1,25g_FGA_T2,25g_FG3M_T1,25g_FG3M_T2,25g_FG3A_T1,25g_FG3A_T2,25g_FTM_T1,25g_FTM_T2,25g_FTA_T1,25g_FTA_T2,25g_OREB_T1,25g_OREB_T2,25g_DREB_T1,25g_DREB_T2,25g_REB_T1,25g_REB_T2,25g_AST_T1,25g_AST_T2,25g_STL_T1,25g_STL_T2,25g_BLK_T1,25g_BLK_T2,25g_TO_T1,25g_TO_T2,25g_PF_T1,25g_PF_T2,25g_PTS_T1,25g_PTS_T2,25g_PLUS_MINUS_T1,25g_PLUS_MINUS_T2,25g_FG_PCT_T1,...,50g_FGM_T2,50g_FGA_T1,50g_FGA_T2,50g_FG3M_T1,50g_FG3M_T2,50g_FG3A_T1,50g_FG3A_T2,50g_FTM_T1,50g_FTM_T2,50g_FTA_T1,50g_FTA_T2,50g_OREB_T1,50g_OREB_T2,50g_DREB_T1,50g_DREB_T2,50g_REB_T1,50g_REB_T2,50g_AST_T1,50g_AST_T2,50g_STL_T1,50g_STL_T2,50g_BLK_T1,50g_BLK_T2,50g_TO_T1,50g_TO_T2,50g_PF_T1,50g_PF_T2,50g_PTS_T1,50g_PTS_T2,50g_PLUS_MINUS_T1,50g_PLUS_MINUS_T2,50g_FG_PCT_T1,50g_FG_PCT_T2,50g_FG3_PCT_T1,50g_FG3_PCT_T2,50g_FT_PCT_T1,50g_FT_PCT_T2,50g_SCORE_DIFF_T1,50g_SCORE_DIFF_T2,50g_WON_T1,50g_WON_T2,75g_FGM_T1,75g_FGM_T2,75g_FGA_T1,75g_FGA_T2,75g_FG3M_T1,75g_FG3M_T2,75g_FG3A_T1,75g_FG3A_T2,75g_FTM_T1,75g_FTM_T2,75g_FTA_T1,75g_FTA_T2,75g_OREB_T1,75g_OREB_T2,75g_DREB_T1,75g_DREB_T2,75g_REB_T1,75g_REB_T2,75g_AST_T1,75g_AST_T2,75g_STL_T1,75g_STL_T2,75g_BLK_T1,75g_BLK_T2,75g_TO_T1,75g_TO_T2,75g_PF_T1,75g_PF_T2,75g_PTS_T1,75g_PTS_T2,75g_PLUS_MINUS_T1,75g_PLUS_MINUS_T2,75g_FG_PCT_T1,75g_FG_PCT_T2,75g_FG3_PCT_T1,75g_FG3_PCT_T2,75g_FT_PCT_T1,75g_FT_PCT_T2,75g_SCORE_DIFF_T1,75g_SCORE_DIFF_T2,75g_WON_T1,75g_WON_T2,100g_FGM_T1,100g_FGM_T2,100g_FGA_T1,100g_FGA_T2,100g_FG3M_T1,100g_FG3M_T2,100g_FG3A_T1,100g_FG3A_T2,100g_FTM_T1,100g_FTM_T2,100g_FTA_T1,100g_FTA_T2,100g_OREB_T1,100g_OREB_T2,100g_DREB_T1,100g_DREB_T2,100g_REB_T1,100g_REB_T2,100g_AST_T1,100g_AST_T2,100g_STL_T1,100g_STL_T2,100g_BLK_T1,100g_BLK_T2,100g_TO_T1,100g_TO_T2,100g_PF_T1,100g_PF_T2,100g_PTS_T1,100g_PTS_T2,100g_PLUS_MINUS_T1,100g_PLUS_MINUS_T2,100g_FG_PCT_T1,100g_FG_PCT_T2,100g_FG3_PCT_T1,100g_FG3_PCT_T2,100g_FT_PCT_T1,100g_FT_PCT_T2,100g_SCORE_DIFF_T1,100g_SCORE_DIFF_T2,100g_WON_T1,100g_WON_T2
0,21400001.0,20141028_nba_Orlando_New Orleans,20141028.0,1.610613e+09,32.0,84.0,4.0,11.0,16.0,21.0,16.0,40.0,56.0,17.0,5.0,9.0,18.0,25.0,84.0,-85.0,0.380952,0.363636,0.761905,-17.0,0.0,1.610613e+09,41.0,101.0,4.0,17.0,15.0,31.0,26.0,36.0,62.0,20.0,10.0,17.0,9.0,17.0,101.0,85.0,0.405941,0.235294,0.483871,17.0,1.0,0.208333,9.00,191.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN